In [1]:
# Import libraries and packages
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
# Load validation cohort
cohort = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models/dataset_agg_sr_inf_phoenix.parquet.gzip')

# Load admitting diagnoses
adm_diag = pd.read_parquet('/labs/collab/ECMO/new_data/TAB5_Admitting_Diagnoses.parquet.gzip')
adm_diag.columns = ['patid', 'name', 'dob', 'mrn', 'csn', 'icd9', 'icd10', 'diagnosis', 'admission_date']
adm_diag.dropna(subset=['diagnosis'], inplace=True)
adm_diag = adm_diag[adm_diag['csn'].isin(cohort['csn'].unique().tolist())]
adm_diag.head()

In [3]:
# Add flags
adm_diag['Sepsis'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('sepsis|septicemia', case=False, na=False), 'Sepsis'] = 1

adm_diag['Septic Shock'] = 0
adm_diag.loc[(adm_diag['diagnosis'].str.contains('septic shock', case=False, na=False)) & 
            ~(adm_diag['diagnosis'].str.contains('septicemia|without', case=False, na=False)), 'Septic Shock'] = 1

adm_diag['Sickle Cell Disease'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('sickle', case=False, na=False), 'Sickle Cell Disease'] = 1

adm_diag['Diabetic Ketoacidosis'] = 0
adm_diag.loc[(adm_diag['diagnosis'].str.contains('diabetes', case=False, na=False)) &
            (adm_diag['diagnosis'].str.contains('ketoacidosis', case=False, na=False)), 'Diabetic Ketoacidosis'] = 1

adm_diag['Asthmaticus'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('asthmaticus', case=False, na=False), 'Asthmaticus'] = 1

adm_diag['Kidney Failure'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('kidney disease|kidney failure', case=False, na=False), 'Kidney Failure'] = 1

In [ ]:
adm_diag.to_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_analysis/adm_diag_predictions.parquet.gzip', compression='gzip')
adm_diag.head()